In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
   # for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

In [2]:
batch_size = 32
IMG_SIZE = (224, 224)
NUM_CLASSES = 4
LR = 0.0001  # Reduced learning rate

In [3]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  
    width_shift_range=0.1,  
    height_shift_range=0.1,  
    shear_range=0.1,  
    zoom_range=0.1,  
    horizontal_flip=True,
)

In [4]:
# Load train and validation data
train_ds = train_datagen.flow_from_directory(
    'dataset7/train',
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 2416 images belonging to 4 classes.


In [5]:
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_ds = valid_datagen.flow_from_directory(
    'dataset7/valid',
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 344 images belonging to 4 classes.


In [6]:
# Define model architecture with fine-tuning and regularization
base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Enable fine-tuning

In [7]:
# Add some regularization
for layer in base_model.layers:
    if isinstance(layer, keras.layers.Conv2D):
        layer.kernel_regularizer = keras.regularizers.l2(0.01)
    elif isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = False  # Fix batch normalization layers during fine-tuning

model = keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),  # Add dropout for regularization
    Dense(NUM_CLASSES, activation='softmax')
])

In [8]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=LR),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [9]:
# Callbacks
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3)
checkpoint_callback = ModelCheckpoint("best_model.keras", monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [10]:
# Train the model
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=valid_ds,
    callbacks=[lr_scheduler, checkpoint_callback, early_stopping_callback]
)

Epoch 1/50


76/76 [==============================] - 165s 2s/step - loss: 1.1011 - accuracy: 0.5265 - val_loss: 1.0761 - val_accuracy: 0.5581 - lr: 1.0000e-04
Epoch 2/50
76/76 [==============================] - 141s 2s/step - loss: 0.9556 - accuracy: 0.5844 - val_loss: 0.9290 - val_accuracy: 0.6047 - lr: 1.0000e-04
Epoch 3/50
76/76 [==============================] - 138s 2s/step - loss: 0.8501 - accuracy: 0.6370 - val_loss: 0.8418 - val_accuracy: 0.6076 - lr: 1.0000e-04
Epoch 4/50
76/76 [==============================] - 140s 2s/step - loss: 0.8302 - accuracy: 0.6407 - val_loss: 0.8940 - val_accuracy: 0.6453 - lr: 1.0000e-04
Epoch 5/50
76/76 [==============================] - 139s 2s/step - loss: 0.7594 - accuracy: 0.6676 - val_loss: 0.8387 - val_accuracy: 0.6250 - lr: 1.0000e-04
Epoch 6/50
76/76 [==============================] - 140s 2s/step - loss: 0.7338 - accuracy: 0.6809 - val_loss: 0.7635 - val_accuracy: 0.6366 - lr: 1.0000e-04
Epoch 7/50
76/76 [==============================] 

In [11]:
# Evaluate the model on test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_ds = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 172 images belonging to 4 classes.


In [12]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test accuracy: {test_accuracy:.4f}")

6/6 [==============================] - 3s 380ms/step - loss: 0.7114 - accuracy: 0.7267
Test accuracy: 0.7267


In [17]:
from tensorflow.keras.models import load_model

In [16]:
model

In [18]:
model.save('glaucoma6.keras')

In [19]:
d = load_model('glaucoma6.keras')
d